<a href="https://colab.research.google.com/github/LCherop/ML_Tasks/blob/main/Copy_of_ml_lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import the required libraries

In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV

### Import the csv and read the csv file using Pandas

In [3]:
housing_df = pd.read_csv(r'/content/modified_data.csv')
housing_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [4]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

### Preparing for cleaning

In [5]:
#Drop the ID column because it will not be useful in training
housing_df.drop('Id',inplace=True,axis=1)

In [7]:
housing_df.head(4)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000


### Fill in missing values

In [18]:
#find out how many columns have missing values
features_with_null=list(housing_df.columns[housing_df.isna().any()])
len(features_with_null)

15

In [19]:
#Calculate percentages of missing values for each column
for feature in features_with_null:
  print(feature,' ',housing_df[feature].isna().sum()/len(housing_df)*100,' ',housing_df[feature].dtype)


LotFrontage   17.73972602739726   float64
MasVnrType   0.547945205479452   object
MasVnrArea   0.547945205479452   float64
BsmtQual   2.5342465753424657   object
BsmtCond   2.5342465753424657   object
BsmtExposure   2.6027397260273974   object
BsmtFinType1   2.5342465753424657   object
BsmtFinType2   2.6027397260273974   object
Electrical   0.0684931506849315   object
FireplaceQu   47.26027397260274   object
GarageType   5.5479452054794525   object
GarageYrBlt   5.5479452054794525   float64
GarageFinish   5.5479452054794525   object
GarageQual   5.5479452054794525   object
GarageCond   5.5479452054794525   object


In [20]:
#Fill in missing values in features with float64 dtype using mean fill
for i in features_with_null:
  if housing_df[i].dtype == 'float64':
    housing_df[i].fillna(housing_df[i].mean(),inplace=True)

In [24]:
#Fill in missing values in features with object dtype using backward fill
for i in features_with_null:
  if housing_df[i].dtype == 'object':
    housing_df[i].fillna(method='bfill',inplace=True)

In [25]:
#Check if any missing values are still present
housing_df.columns[housing_df.isna().any()]

Index(['FireplaceQu'], dtype='object')

In [26]:
#Fill in missing value in FireplaceQu using ffil
housing_df['FireplaceQu'].fillna(method='ffill',inplace=True)

### Encode the Dataset

In [28]:
#Have all features in a list
all_features=list(housing_df.columns)
len(all_features)

76

In [27]:
#Create list of all categorical features
categorical_features=list(housing_df.select_dtypes(include=['object']).columns)
len(categorical_features)

39

In [29]:
#Create list of all non-categorical features
num_features = list(set(all_features)-set(categorical_features))
len(num_features)

37

In [51]:
target = ['SalePrice']

In [30]:
#Split categorical features into nominal and ordinal
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [31]:
#Encode ordinal features
for j in ordinal:
  housing_df[feature]=housing_df[feature].astype('category').cat.codes

#Assign encoded ordinal data to a data frame
df_ordinal=housing_df[ordinal]

In [33]:
#Encode nominal features and assign to data frame
df_nominal=pd.get_dummies(housing_df[nominal])

In [35]:
#Assign numerical data to a data frame
housing_df[num_features]

,Fireplaces,2ndFlrSF,BsmtFullBath,MoSold,GrLivArea,PoolArea,1stFlrSF,BsmtUnfSF,KitchenAbvGr,TotRmsAbvGrd,...,GarageYrBlt,HalfBath,YrSold,ScreenPorch,LotFrontage,YearBuilt,LotArea,YearRemodAdd,LowQualFinSF,WoodDeckSF
0,0,854,1,2,1710,0,856,150,1,8,...,2003.0,1,2008,0,65.0,2003,8450,2003,0,0
1,1,0,0,5,1262,0,1262,284,1,6,...,1976.0,0,2007,0,80.0,1976,9600,1976,0,298
2,1,866,1,9,1786,0,920,434,1,6,...,2001.0,1,2008,0,68.0,2001,11250,2002,0,0
3,1,756,1,2,1717,0,961,540,1,7,...,1998.0,0,2006,0,60.0,1915,9550,1970,0,0
4,1,1053,1,12,2198,0,1145,490,1,9,...,2000.0,1,2008,0,84.0,2000,14260,2000,0,192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1,694,0,8,1647,0,953,953,1,7,...,1999.0,1,2007,0,62.0,1999,7917,2000,0,0
1456,2,0,1,2,2073,0,2073,589,1,7,...,1978.0,0,2010,0,85.0,1978,13175,1988,0,349
1457,2,1152,0,5,2340,0,1188,877,1,9,...,1941.0,0,2010,0,66.0,1941,9042,2006,0,0
1458,0,0,1,4,1078,0,1078,0,1,5,...,1950.0,0,2010,0,68.0,1950,9717,1996,0,366


In [36]:
#Concantenate all encoded data to a new dataframe
encoded_data = pd.concat([df_nominal,df_ordinal,housing_df[num_features]],axis=1)

In [38]:
encoded_data.shape

(1460, 107)

### Standardize the dataset

In [66]:
encoded_data.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,...,GarageYrBlt,HalfBath,YrSold,ScreenPorch,LotFrontage,YearBuilt,LotArea,YearRemodAdd,LowQualFinSF,WoodDeckSF
0,0,0,0,1,0,0,0,0,1,0,...,2003.0,1,2008,0,65.0,2003,8450,2003,0,0
1,0,0,0,1,0,0,0,0,1,0,...,1976.0,0,2007,0,80.0,1976,9600,1976,0,298
2,0,0,0,1,0,0,0,0,1,0,...,2001.0,1,2008,0,68.0,2001,11250,2002,0,0
3,0,0,0,1,0,0,0,0,1,0,...,1998.0,0,2006,0,60.0,1915,9550,1970,0,0
4,0,0,0,1,0,0,0,0,1,0,...,2000.0,1,2008,0,84.0,2000,14260,2000,0,192


In [52]:
#Drop column with the target from the encoded data
encoded_data.drop('SalePrice',axis=1,inplace=True)

In [54]:
#define features(x) and target(y)
x = encoded_data.to_numpy()
y = housing_df[target]
y = y.to_numpy()

In [57]:
x.shape

(1460, 106)

In [ ]:
X = StandardScaler().fit_transform(x)

In [ ]:
y = StandardScaler().fit_transform(y)


### Feature Extraction using PCA

In [61]:
pca = PCA(n_components=2)

In [ ]:
p_components=pca.fit_transform(x)

In [ ]:
pca.explained_variance_ratio_

### Feature Extraction using L1

### Training the Regression Model

In [64]:
reg=LassoCV()

In [ ]:
reg.fit(p_components,y)

In [ ]:
reg.coef_